# Ejemplo 4: Politics Debate
En el presente ejemplo se implementará un sistema multi agente haciendo uso del patrón de diseño multi agente llamado Reflection. Por tal motivo, se crearán los siguientes agentes:

- RightWingAgent: Este agente representará posiciones conservadoras o liberales en términos económicos según el tema de debate.
- LeftWingAgentt: Este agente representará posiciones progresistas o de izquierda según el tema de debate.

In [1]:
# Load environment variables

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from dataclasses import dataclass


@dataclass
class RightWingTask:
    task: str

@dataclass
class LeftWingTask:
    session_id: str
    right_wing_task: str
    argument: str


@dataclass
class LeftWingResult:
    review: str
    session_id: str
    consent: bool

In [4]:
import uuid
from typing import Dict, List

from autogen_core import MessageContext, RoutedAgent, TopicId, default_subscription, message_handler
from autogen_core.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
)

In [5]:
@default_subscription
class RightWingAgent(RoutedAgent):
    """An agent that express opinions from right wing side."""

    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("A right wing agent.")
        # Counter to limit the debate
        self.max_turns = 6
        self.current_turns = 0
        
        self._system_messages: List[LLMMessage] = [
            SystemMessage(
                content="""Eres un experto en política y filosofía con una perspectiva conservadora y de derecha.
Tu tarea es debatir sobre el tema que provea el usuario contra un oponente que representa una visión de izquierda.

Reglas del debate:

Defiende tu posición de manera estructurada y con argumentos sólidos.
Usa evidencia histórica, económica o política para respaldar tu postura.
Mantén un tono respetuoso, pero desafiante.
Refuta los puntos de tu oponente con lógica y contraejemplos.
No cedas en tus convicciones; tu objetivo es persuadir y demostrar la superioridad de tu argumento.
Tu argumento debe ser de no más de 50 palabras.
""",
            )
        ]
        self._model_client = model_client
        self._session_memory: Dict[str, List[RightWingTask | LeftWingTask | LeftWingResult]] = {}

    @message_handler
    async def handle_right_wing_task(self, message: RightWingTask, ctx: MessageContext) -> None:
        # Store the messages in a temporary memory for this request only.
        session_id = str(uuid.uuid4())
        self._session_memory.setdefault(session_id, []).append(message)
        # Generate a response using the chat completion API.
        response = await self._model_client.create(
            self._system_messages + [UserMessage(content=message.task, source=self.metadata["type"])],
            cancellation_token=ctx.cancellation_token,
        )
        assert isinstance(response.content, str)
        print(f"RIGHT WING: {response.content}")
        # Create a left wing task.
        left_wing_task = LeftWingTask(
            session_id=session_id,
            right_wing_task=message.task,
            argument=response.content,
        )
        # Store the left wing task in the session memory.
        self._session_memory[session_id].append(left_wing_task)
        # Publish a code review task.
        await self.publish_message(left_wing_task, topic_id=TopicId("default", self.id.key))
        self.current_turns += 1

    @message_handler
    async def handle_left_wing_result(self, message: LeftWingResult, ctx: MessageContext) -> None:
        self.current_turns += 1
        # Store the review result in the session memory.
        self._session_memory[message.session_id].append(message)
        # Obtain the request from previous messages.
        left_request = next(
            m for m in reversed(self._session_memory[message.session_id]) if isinstance(m, LeftWingTask)
        )
        assert left_request is not None
        # Check if the code is approved.
        if message.consent  or self.current_turns >= self.max_turns:
            print("*********************************")
            print("El debate ha terminado")
            print("*********************************")
        else:
            # Create a list of LLM messages to send to the model.
            messages: List[LLMMessage] = [*self._system_messages]
            for m in self._session_memory[message.session_id]:
                if isinstance(m, LeftWingResult):
                    messages.append(UserMessage(content=m.review, source="LeftWing"))
                elif isinstance(m, LeftWingTask):
                    messages.append(AssistantMessage(content=m.argument, source="RightWing"))
                elif isinstance(m, RightWingTask):
                    messages.append(UserMessage(content=m.task, source="User"))
                else:
                    raise ValueError(f"Unexpected message type: {m}")
            # Generate a revision using the chat completion API.
            response = await self._model_client.create(messages, cancellation_token=ctx.cancellation_token)
            assert isinstance(response.content, str)

            print(f"RIGHT WING: {response.content}")
            
            # Create a new code review task.
            left_wing_task = LeftWingTask(
                session_id=message.session_id,
                right_wing_task=left_request.right_wing_task,
                argument=response.content,
            )
            # Store the code review task in the session memory.
            self._session_memory[message.session_id].append(left_wing_task)
            # Publish a new code review task.
            await self.publish_message(left_wing_task, topic_id=TopicId("default", self.id.key))
            self.current_turns += 1


In [6]:
@default_subscription
class LeftWingAgent(RoutedAgent):
    """An agent that express opinions from left wing side."""

    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("A left wing agent.")
        self._system_messages: List[LLMMessage] = [
            SystemMessage(
                content="""Eres un experto en política y filosofía con una perspectiva progresista y de izquierda.
Tu tarea es debatir sobre el tema que provea el usuario contra un oponente que representa una visión de derecha.

Reglas del debate:

Defiende tu posición con argumentos bien estructurados y respaldados por evidencia histórica, económica o política.
Explica los beneficios de tu ideología y por qué es la mejor opción en el tema discutido.
Mantén un tono respetuoso, pero desafiante.
Refuta los argumentos de tu oponente con lógica y datos.
No cedas en tus convicciones; tu objetivo es persuadir y demostrar la superioridad de tu postura.
Tu argumento debe ser de no más de 50 palabras.
""",
            )
        ]
        self._session_memory: Dict[str, List[LeftWingTask | LeftWingResult]] = {}
        self._model_client = model_client

    @message_handler
    async def handle_left_wing_task(self, message: LeftWingTask, ctx: MessageContext) -> None:
        # Format the prompt for the code review.
        # Gather the previous feedback if available.
        previous_feedback = ""
        if message.session_id in self._session_memory:
            previous_review = next(
                (m for m in reversed(self._session_memory[message.session_id]) if isinstance(m, LeftWingResult)),
                None,
            )
            if previous_review is not None:
                previous_feedback = previous_review.review
        # Store the messages in a temporary memory for this request only.
        self._session_memory.setdefault(message.session_id, []).append(message)
        prompt = f"""El tema a debatir es: {message.right_wing_task}
El argumento de tu oponente es:
```
{message.argument}
```

Feedback previo:
{previous_feedback}

Genera un contraargumento a esa posición. Si consideras que estas de acuerdo con la postura de tu oponente, responde con la palabra CONCUERDO.
"""
        # Generate a response using the chat completion API.
        response = await self._model_client.create(
            self._system_messages + [UserMessage(content=prompt, source=self.metadata["type"])],
            cancellation_token=ctx.cancellation_token
        )
        assert isinstance(response.content, str)
        
        print(f"LEFT WING: {response.content}")

        review = response.content
        consent = True if "CONCUERDO" in review else False
        result = LeftWingResult(
            review=review,
            session_id=message.session_id,
            consent=consent,
        )
        # Store the review result in the session memory.
        self._session_memory[message.session_id].append(result)
        # Publish the review result.
        await self.publish_message(result, topic_id=TopicId("default", self.id.key))

In [35]:
import logging

logging.basicConfig(level=logging.WARNING)
logging.getLogger("autogen_core").setLevel(logging.DEBUG)

In [8]:
from autogen_core import DefaultTopicId, SingleThreadedAgentRuntime
from autogen_ext.models.openai import OpenAIChatCompletionClient

runtime = SingleThreadedAgentRuntime()
await LeftWingAgent.register(
    runtime, "LeftWingAgent", lambda: LeftWingAgent(model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"))
)
await RightWingAgent.register(
    runtime, "RightWingAgent", lambda: RightWingAgent(model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"))
)

topic = "Que opinas sobre las subvenciones para el transporte público?"
print(f"Pregunta del debate: {topic}")

runtime.start()
await runtime.publish_message(
    message=RightWingTask(task=topic),
    topic_id=DefaultTopicId(),
)

# Keep processing messages until idle.
await runtime.stop_when_idle()

Pregunta del debate: Que opinas sobre las subvenciones para el transporte público?
RIGHT WING: Las subvenciones para el transporte público suelen distorsionar el mercado, generando una dependencia económica y una mala gestión de recursos. En cambio, promover la competencia y la inversión privada mejora la calidad del servicio, fomentando un transporte más eficiente y sostenible, como han demostrado casos en diversas ciudades con éxito.
LEFT WING: Las subvenciones para el transporte público son esenciales para garantizar accesibilidad y equidad, especialmente para las comunidades vulnerables. Mientras que el mercado por sí solo puede favorecer a los más ricos, una gestión pública adecuada, respaldada por subvenciones, potencia la calidad y eficiencia del servicio, como lo evidencian ciudades con sistemas públicos exitosos.
RIGHT WING: Si bien es cierto que el transporte público debe ser accesible, las subvenciones no siempre logran los resultados deseados. Muchas veces generan ineficien